In [1]:
import os
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

# Login

In [ ]:
# ---- PLACEHOLDERS ----
SPOTIPY_CLIENT_ID = ""
SPOTIPY_CLIENT_SECRET = ""

# This MUST match what you set in your Spotify Developer Dashboard for your app
SPOTIPY_REDIRECT_URI = "http://127.0.0.1:8888/callback"

# Scopes needed:
# - user-library-read: read Liked Songs / Saved Tracks
# - playlist-read-private: read your private playlists
# - playlist-read-collaborative: read collaborative playlists you can access
SCOPE = "user-library-read playlist-read-private playlist-read-collaborative"

sp = Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope=SCOPE,
    open_browser=True
))

me = sp.current_user()
print("Logged in as:", me.get("display_name"), "| user_id:", me.get("id"))

Logged in as: jeffchoize | user_id: jeffchoize


In [3]:
def paginate_items(first_page: dict, item_key: str = "items"):
    """
    Spotify API responses usually look like:
    { items: [...], next: 'url or None', ... }
    Spotipy provides sp.next(page) to fetch subsequent pages.
    """
    page = first_page
    while True:
        for it in page.get(item_key, []):
            yield it
        if page.get("next"):
            page = sp.next(page)
        else:
            break

# Liked Songs

In [5]:
def liked_songs_df(limit=50):
    first = sp.current_user_saved_tracks(limit=limit, offset=0)  # /me/tracks  [oai_citation:4‡Spotify for Developers](https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks?utm_source=chatgpt.com)
    rows = []
    for item in paginate_items(first, "items"):
        t = item.get("track") or {}
        if not t:  # sometimes null
            continue

        artists = ", ".join(a["name"] for a in t.get("artists", []))
        album = (t.get("album") or {}).get("name")
        release_date = (t.get("album") or {}).get("release_date")

        rows.append({
            "source": "liked_songs",
            "source_playlist": "Liked Songs",
            "track_name": t.get("name"),
            "artists": artists,
            "album": album,
            "release_date": release_date,
            "spotify_url": (t.get("external_urls") or {}).get("spotify"),
        })

    return pd.DataFrame(rows)

df_liked = liked_songs_df(10)
df_liked.head(), len(df_liked)

(        source source_playlist                      track_name  \
 0  liked_songs     Liked Songs  West End Girls - 2018 Remaster   
 1  liked_songs     Liked Songs   All Quiet On The Frontal Lobe   
 2  liked_songs     Liked Songs                     Foolish Man   
 3  liked_songs     Liked Songs                     Hello World   
 4  liked_songs     Liked Songs                      Slow Break   
 
                 artists                                              album  \
 0         Pet Shop Boys  Please: Further Listening 1984 - 1986 (2018 Re...   
 1  ||||||||||||||||||||                      All Quiet On The Frontal Lobe   
 2                Jack J                                        Blue Desert   
 3               dBridge                                        Hello World   
 4                 Torus                                     Summer of Love   
 
   release_date                                        spotify_url  
 0   1986-03-24  https://open.spotify.com/track/2Di

# Playlist

In [13]:
def my_playlists():
    first = sp.current_user_playlists(limit=50, offset=0)  # /me/playlists  [oai_citation:5‡Spotify for Developers](https://developer.spotify.com/documentation/web-api/reference/get-a-list-of-current-users-playlists?utm_source=chatgpt.com)
    playlists = []
    for pl in paginate_items(first, "items"):
        if pl["owner"]["id"] == 'jeffchoize':
            playlists.append({
                "playlist_id": pl.get("id"),
                "playlist_name": pl.get("name"),
                "owner_id": (pl.get("owner") or {}).get("id"),
                "public": pl.get("public"),
                "collaborative": pl.get("collaborative"),
                "tracks_total": (pl.get("tracks") or {}).get("total"),
            })
    return pd.DataFrame(playlists)

playlists = my_playlists()
len(playlists)

54

In [14]:
playlists

,playlist_id,playlist_name,owner_id,public,collaborative,tracks_total
0,6P8RM1FHglFUAt9lNSREzE,revive inspo,jeffchoize,False,False,8
1,3W2MRkWLeu5cNvkggN6Qvi,2025上,jeffchoize,True,False,63
2,6Xfxz5x1esA26SQB5Yo3Nm,quiet avant-garde tuesday afternoon (2),jeffchoize,True,False,50
3,1pGDklI9TjMdTtBIHOcBTe,shot of bittersweet,jeffchoize,False,False,11
4,1q3E8ykuuctV9CDPbqEQkq,RHow,jeffchoize,False,False,12
5,2vmdORk0mJV1LUyktpRXtR,seems nice,jeffchoize,False,False,151
6,5wVOwGyBlT5rPwWTSqTj53,Vampire State,jeffchoize,True,False,139
7,4AXRu49y3m2HqhsUcErSl9,My Playlist #47,jeffchoize,True,False,7
8,2wyF7Poewtx05AomiYvD0r,SonicDict,jeffchoize,False,False,64
9,2lfLW8qjQ80BGH1r8wci06,202E,jeffchoize,False,False,13


In [ ]:
def playlist_tracks_df(playlist_id: str, playlist_name: str, limit=100):
    # Get Playlist Items  [oai_citation:6‡Spotify for Developers](https://developer.spotify.com/documentation/web-api/reference/get-playlists-tracks?utm_source=chatgpt.com)
    first = sp.playlist_items(
        playlist_id,
        limit=limit,
        offset=0,
        additional_types=("track",)
    )

    rows = []
    for item in paginate_items(first, "items"):
        t = item.get("track") or {}
        # skip non-tracks (episodes) or removed items
        if not t or t.get("type") != "track":
            continue

        artists = ", ".join(a["name"] for a in t.get("artists", []))
        album = (t.get("album") or {}).get("name")
        release_date = (t.get("album") or {}).get("release_date")

        rows.append({
            "source": "liked_songs",
            "source_playlist": "Liked Songs",
            "track_name": t.get("name"),
            "artists": artists,
            "album": album,
            "release_date": release_date,
            "spotify_url": (t.get("external_urls") or {}).get("spotify"),
        })

    return pd.DataFrame(rows)

# Pull all playlists (can take a bit if you have many)
dfs_by_playlist = {}
for pl in playlists:
    pid, pname = pl["playlist_id"], pl["playlist_name"]
    dfs_by_playlist[pname] = playlist_tracks_df(pid, pname)

sum(len(df) for df in dfs_by_playlist.values()), list(dfs_by_playlist.keys())[:5]

# Album

In [15]:
def get_saved_albums_df(limit=50):
    """
    Returns a DataFrame of your saved albums (Library > Albums).
    """
    first = sp.current_user_saved_albums(limit=limit, offset=0)
    rows = []

    for item in paginate_items(first, "items"):
        album = item.get("album") or {}
        if not album:
            continue

        artists = ", ".join(a["name"] for a in album.get("artists", []))
        spotify_url = (album.get("external_urls") or {}).get("spotify")

        rows.append({
            "added_at": item.get("added_at"),
            "album_name": album.get("name"),
            "artist": artists,
            "release_date": album.get("release_date"),
            "album_type": album.get("album_type"),          # album / single / compilation
            "total_tracks": album.get("total_tracks"),
            "spotify_url": spotify_url,
        })

    return pd.DataFrame(rows)

df_saved_albums = get_saved_albums_df()

In [16]:
len(df_saved_albums)

187

In [19]:
df_saved_albums.sort_values('added_at', ascending=False)

,added_at,album_name,artist,release_date,album_type,total_tracks,spotify_url
0,2026-01-27T14:31:42Z,Blue Desert,Jack J,2024-11-01,album,11,https://open.spotify.com/album/1H96rR2NE8XkXZw...
4,2026-01-10T20:55:35Z,Mekong Ballad,COLA REN,2025-11-05,single,5,https://open.spotify.com/album/35BgR7SFHZQVTid...
28,2025-12-16T04:32:31Z,24/7 Heaven,trickpony,2025-12-15,album,6,https://open.spotify.com/album/4cyKKwiZ9g85rxT...
18,2025-12-14T18:22:51Z,Algorithmic Accompaniment,"Nueen, Jason Kolàr",2025-08-19,album,12,https://open.spotify.com/album/5sEmK4xcdrfcKOg...
29,2025-12-13T02:32:04Z,Lazarus (Adult Swim Original Series Soundtrack),Kamasi Washington,2025-04-11,album,11,https://open.spotify.com/album/5gD3tPSWfnzu1XW...
...,...,...,...,...,...,...,...
182,2018-07-13T08:45:55Z,Where Owls Know My Name,Rivers of Nihil,2018-03-16,album,10,https://open.spotify.com/album/6Ue1iqByWrHTVFU...
183,2018-06-11T02:02:36Z,II,Vale Of Pnath,2016-06-10,album,8,https://open.spotify.com/album/6DmxTvjRTX1X3oM...
184,2018-04-11T05:55:09Z,Augment,ERRA,2013-10-29,album,12,https://open.spotify.com/album/0SHApkUnsJkdt6a...
185,2018-04-04T05:45:42Z,Lost Forever // Lost Together,Architects,2014-03-11,album,11,https://open.spotify.com/album/6ut947CLkGJcmiF...


In [20]:
from pathlib import Path

out_xlsx = Path("spotify_library_export.xlsx")

with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    # Sheet 1: liked songs
    df_liked.to_excel(writer, sheet_name="Liked Songs", index=False)

    df_saved_albums.to_excel(writer, sheet_name="Saved Album", index=False)

    # Other sheets: playlists
    # for name, df in dfs_by_playlist.items():
    #     # Excel sheet name max length is 31 chars; also avoid illegal chars
    #     safe_name = name.replace("/", "_")[:31] or "Playlist"
    #     df.to_excel(writer, sheet_name=safe_name, index=False)

out_xlsx

PosixPath('spotify_library_export.xlsx')